In [1]:
import tweepy
import configparser
import pandas as pd
from twitter_authentication import bearer_token
import time

In [2]:
# read configs
config = configparser.ConfigParser()
config.read('config.ini')

api_key = config['twitter']['api_key']
api_key_secret = config['twitter']['api_key_secret']

access_token = config['twitter']['access_token']
access_token_secret = config['twitter']['access_token_secret']

client = tweepy.Client(bearer_token, wait_on_rate_limit = True)

In [3]:
hoax_tweets = []
for response in tweepy.Paginator(client.search_all_tweets, 
                                 query = 'seattle -is:retweet lang:en',
                                 user_fields = ['username', 'public_metrics', 'description', 'location'],
                                 tweet_fields = ['created_at', 'geo', 'public_metrics', 'text'],
                                 expansions = 'author_id',
                                 start_time = '2021-03-01T00:00:00Z',
                                 end_time = '2021-03-21T00:00:00Z',
                              max_results=100, limit = 1):
    time.sleep(1)
    hoax_tweets.append(response)

In [4]:
hoax_tweets[0].data[0]

<Tweet id=1373424040287875073 text=@Paradisitys Hope Seattle don't lose momentum now though, and hope they fry to 2nd/3rd place at least. Maybe even 1st if they somehow pull one over Optic and 3-0/3-1 everyone else>

In [5]:
result = []
user_dict = {}
for response in hoax_tweets:
    for user in response.includes['users']:
        user_dict[user.id] = {'username': user.username, 
                              'followers': user.public_metrics['followers_count'],
                              'tweets': user.public_metrics['tweet_count'],
                              'description': user.description,
                              'location': user.location
                             }
    for tweet in response.data:
        author_info = user_dict[tweet.author_id]
        result.append({'author_id': tweet.author_id,
                       'author_location': author_info['location'],
                       'text': tweet.text,
                       'created_at': tweet.created_at,
                       'geo': tweet.geo
                      })

df = pd.DataFrame(result)


In [6]:
print(df)

              author_id    author_location  \
0   1167036175397396480               None   
1             844283084        Seattle, WA   
2   1190149526167916544  Pennsylvania, USA   
3            2570624584               None   
4             127137318        Seattle, WA   
..                  ...                ...   
95           2895745954             Philly   
96  1135329063525072901    Chattanooga, TN   
97  1025914937225486336               None   
98            189590587               None   
99  1264361875254530048         Queens, NY   

                                                 text  \
0   @Paradisitys Hope Seattle don't lose momentum ...   
1   What's new on the Happy Shaadi blog? A whole n...   
2   bathroom stall in seattle - 1:31 \n\n(If you u...   
3   Loved @TheDoubleclicks playing for @dw_seattle...   
4        @chris_andre1 @Mitch_Seattle Best answer yet   
..                                                ...   
95  Really good series out of Seattle they could

In [7]:
df.to_csv('test.csv', encoding='utf-8', index=False)